In [1]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [1]:
cd "drive/My Drive/Colab Notebooks/ML_proj2"

/content/drive/My Drive/Colab Notebooks/ML_proj2


In [2]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image

from keras.preprocessing.image import ImageDataGenerator
from skimage import io

import tensorflow as tf
from tensorflow import keras

from sklearn.feature_extraction import image

#import kerastuner as kt

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tensorflow.keras import regularizers

from preprocess_data import *

In [4]:
root_dir =os.path.join("data", "training")

X_train,X_test,Y_train,Y_test= preprocess(root_dir, max_iters=30 , random_state=0,
                              test_ratio=0.2, foreground_threshold=0.25)

here 2


In [ ]:
#imlement CNN
INPUT_SHAPE=(X_train.shape[1],X_train.shape[2],X_train.shape[3])

#Input layer
INPUT = keras.layers.Input(shape=INPUT_SHAPE)

pool0=keras.layers.AveragePooling2D(pool_size=(2,2),
                                    strides=(1,1))(INPUT)
                                    
conv1 =keras.layers.Conv2D(filters=128,
                           kernel_size=(5,5),
                           strides=(1,1),
                           activation='relu',
                           padding='same',
                        input_shape=INPUT_SHAPE)(pool0)

pool1 = keras.layers.MaxPool2D(pool_size=(2,2),
                               padding='same')(conv1)
drop1 = keras.layers.Dropout(0.2)(pool1)

conv2 =  keras.layers.Conv2D(filters=256,
                           kernel_size=(5,5),
                           strides=(2,2),
                           activation='relu',
                           padding='same',
                        input_shape=INPUT_SHAPE)(drop1)    
pool2 = keras.layers.MaxPool2D(pool_size=(2,2),
                               padding='same')(conv2)
drop2 = keras.layers.Dropout(0.3)(pool2)


conv3 =  keras.layers.Conv2D(filters=64,
                           kernel_size=(10,10),
                           strides=(2,2),
                           activation='relu',
                           padding='same',
                        input_shape=INPUT_SHAPE)(drop2)  
                        
pool3 = keras.layers.MaxPool2D(pool_size=(2,2),
                               padding='same')(drop2)
drop3 = keras.layers.Dropout(0.3)(pool3)

flat = keras.layers.Flatten()(drop3)

dense1 = keras.layers.Dense(32,
                            kernel_regularizer=tf.keras.regularizers.l1(0.01),
                            activation='softmax')(flat)

dense2 = keras.layers.Dense(2,
                            kernel_regularizer=tf.keras.regularizers.l1(0.01),
                            activation='sigmoid')(dense1)

model = keras.Model(inputs=INPUT,outputs=dense2)
print(model.summary())
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

In [ ]:
model.fit(X_train, Y_train, epochs=3, batch_size=64)

In [ ]:
test=model.evaluate(X_test,Y_test)